In [6]:
from DataManagement.data_manager import DataManager
from DataManagement.coin_data import CoinData

import pandas as pd
import numpy as np

In [7]:
data_manager = DataManager()

In [9]:
coin_symbol_list = ["BTCUSDT", "ETHUSDT"]
feature = "close_pct"
kline_size = "1h"

In [21]:
coin_data_list = [data_manager.get_historical_data_CoinData(coin_symbol, kline_size="1h") for coin_symbol in coin_symbol_list]

In [43]:
btc = coin_data_list[0].df
eth = coin_data_list[1].df

In [45]:
eth.index.values

array(['2017-08-17T04:00:00.000000000', '2017-08-17T05:00:00.000000000',
       '2017-08-17T06:00:00.000000000', ...,
       '2021-06-24T07:00:00.000000000', '2021-06-24T08:00:00.000000000',
       '2021-06-24T09:00:00.000000000'], dtype='datetime64[ns]')

# Join close_pct columns of all coins togther

In [42]:
df = pd.DataFrame()
first_coin_data = coin_data_list[0]
df[f"{feature}_{first_coin_data.coin_symbol}"] = first_coin_data.df["close_pct"]
# df["close_pct"].rename()
for coin_data in coin_data_list[1:]:
    df = df.join(coin_data.df[feature], how="outer", rsuffix="uee", on="timestamp")
# df_list[0].df[["close_pct", "high"]].join(df_list[1].df["close_pct"], how="outer", lsuffix="yeet")
df

,timestamp,close_pct_BTCUSDT,close_pct
2017-08-17 04:00:00,2017-08-17 04:00:00.000,NaN,NaN
2017-08-17 05:00:00,2017-08-17 05:00:00.000,0.001506,0.004940
2017-08-17 06:00:00,2017-08-17 06:00:00.000,0.002093,-0.001386
2017-08-17 07:00:00,2017-08-17 07:00:00.000,0.005929,0.017444
2017-08-17 08:00:00,2017-08-17 08:00:00.000,0.002460,0.002143
...,...,...,...
NaT,2018-02-10 23:28:14.800,NaN,-0.018972
NaT,2018-02-11 00:28:14.800,NaN,-0.016023
NaT,2018-02-11 01:28:14.800,NaN,0.005859
NaT,2018-02-11 02:28:14.800,NaN,-0.025485


In [16]:
df_list[0].df.join(other=[cd.df for cd in df_list], on="index")

ValueError: Joining multiple DataFrames only supported for joining on index

In [ ]:
# min_daterange = np.argmin(list(map(lambda cd: len(cd.df), df_list)))

idx = df_list[0].df.index.intersection(other=[cd.df.index for cd in df_list[1:]])
print(len(idx))
# df_list[0].df.iloc[idx]

NameError: name 'df_list' is not defined

In [7]:
df = pd.DataFrame(columns=[symbol + feature for symbol in coin_symbol_list])
for cd in df_list:
    col_name = cd.coin_symbol + feature
    df[col_name] = cd.df[feature]

ValueError: cannot reindex from a duplicate axis